In [1]:
import nd2
import numpy as np
import tifffile
import pandas as pd
import matplotlib.pyplot as plt
import os.path

In [2]:
def shift_channel_zstack(imarray, channel_toshift, shift_pixel, shift_sign = True, leftright = True):
    newim = np.zeros(np.shape(imarray))
    total_dim = len(np.shape(imarray))
    if (total_dim == 4):
        print('Z stack multi channel image detected, starting channel shifting...')
    else:
        print('Image demension wrong, check if it is multi channel')
        return
    # print(np.shape(imarray))
    # print(channel_array)
    # print(channel_toshift)

    channel_array = [x for x in range(0, np.shape(imarray)[1])]
    unchanged = np.delete(channel_array, channel_toshift-1)

    # #copy channels that are not supposed to shift33
    for i in unchanged:
        newim[:, i, :, :] = imarray[:, i, :, :]

    #now shift
    if (leftright == True):
        #shift on x axis
        max_shift_dim = np.shape(imarray)[3]
        if (shift_sign):
            newim[:, channel_toshift-1, : , shift_pixel:] = imarray[:, channel_toshift-1,
                                                            :, :max_shift_dim-shift_pixel]
        else:
            newim[:, channel_toshift-1, :, :max_shift_dim-shift_pixel] = imarray[
                :, channel_toshift-1, :, shift_pixel:
            ]
    else:
        #shift on y axis
        max_shift_dim = np.shape(imarray)[2]
        # print('max: '+str(max_shift_dim))
        # print('shift p:'+str(shift))
        if (shift_sign):
            newim[:, channel_toshift-1, shift_pixel:, :] = imarray[
                :, channel_toshift-1, :max_shift_dim-shift_pixel, :]
        else:
            newim[:, channel_toshift-1, :max_shift_dim-shift_pixel, :] = imarray[
                :, channel_toshift-1, shift_pixel:, :]
    print('filtering finished!')
    return newim

In [19]:
impath =r"D:\localData\trace_pipe_DJ\4088\rgc_spd\c9\c9_408_488_594_120225im025.nd2"
im = nd2.imread(impath)
#print(np.shape(im))
newim = shift_channel_zstack(im,3,33, leftright = False)

Z stack multi channel image detected, starting channel shifting...
filtering finished!


In [20]:
filename = os.path.basename(impath)
directory = os.path.dirname(impath)
file_nameparts = filename.split('.')
file_nameparts[0] = file_nameparts[0]+'_corrected'
file_nameparts[1] = 'tif'
filename_new = '.'.join(file_nameparts)
newpath = os.path.join(directory, filename_new)
tifffile.imwrite(newpath, newim)
